In [1]:
# -*- config : utf-8 -*-
import numpy as np
import tensorflow as tf
import sys
import time as ti
import warnings

# 경고 메시지 무시
warnings.filterwarnings(action='ignore')

In [2]:
# --- 분석 수행 및 결과 저장 클래스 선언 ---
class EsNoiseAnalyzer:
    
    def __init__(self, modeldir):
        self.modeldir = modeldir
        self.elapsed_time = 0
        self.predict_list = []
        
    def analysis(self, input_list):
        with tf.Session(graph=tf.Graph()) as sess:
            print("START : " + self.modeldir)
            
            # 모델 로딩
            tf.saved_model.loader.load(sess, ["ver1"], self.modeldir)
            
            graph = tf.get_default_graph()
            
            x = graph.get_tensor_by_name("in:0")
            out = graph.get_tensor_by_name("out:0")
            
            # 모델 분석 수행
            start_time = ti.time()
            _out = sess.run([out], feed_dict={x:input_list})
            end_time = ti.time()
            
            self.elapsed_time = end_time - start_time
            
            # 분석 결과 저장
            for index in range(0, input_list.shape[0]):
                self.predict_list.append(np.argmax(_out[0][index], axis=0))
                
            print("END : " + self.modeldir)

In [3]:
# --- 분석할 데이터 로딩 ---
# 데이터 구조
# noise_list : 학습용 데이터(이미지 개수, 48, 64, 3)
# noise_label_list : 학습용 라벨 데이터(이미지 개수, 18)

# 학습 데이터 파일명
noise_list_file = "/src/hyebin/esnoise/noise_spec_data.npy"
# 라벨 데이터 파일명
noise_label_list_file = "/src/hyebin/esnoise/noise_label_data.npy"

print("load noise data : ", noise_list_file)
noise_list = np.load(noise_list_file, allow_pickle=True)
print("load complete")

print("load noise label data : ", noise_label_list_file)
noise_label_list = np.load(noise_label_list_file, allow_pickle=True)
print("load complete")

print(noise_list.shape)
print(noise_label_list.shape)

load noise data :  /src/hyebin/esnoise/noise_spec_data.npy
load complete
load noise label data :  /src/hyebin/esnoise/noise_label_data.npy
load complete
(18000, 48, 64, 3)
(18000,)


In [4]:
# --- 분석 데이터에서 분석 수행용 데이터 일부 추출 ---
# 분석 데이터 설정
# 분석 데이터 중 최초 데이터 위치
base_index = 10000
# 분석할 데이터의 개수
analysis_count = 5000

# 분석용 데이터와 라벨
analysis_x = noise_list[base_index:base_index + analysis_count]
analysis_y = noise_label_list[base_index:base_index + analysis_count]

In [5]:
# --- 분석 모델 로딩 및 분석 수행 ---
# 소음 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/NOISE-SPEC-01"

analyzer = EsNoiseAnalyzer(modeldir)
analyzer.analysis(analysis_x)

START : /src/hyebin/model/NOISE-SPEC-01
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /src/hyebin/model/NOISE-SPEC-01/variables/variables
END : /src/hyebin/model/NOISE-SPEC-01


In [6]:
# --- 분석 결과 출력 ---
# 정답 갯수
correct_count = 0
# 오답 갯수
incorrect_count = 0

print("=======================================")
print("index\tpredict\tlabel\tcorrect")
print("---------------------------------------")

for index in range(analysis_count):
    # 예측값 
    predict = analyzer.predict_list[index]
    # 실제값
    label = analysis_y[index]
    
    result = "0"
    
    # 정답일 경우, 정답 카운트 1 증가
    # 오답일 경우, 오답 카운트 1 증가 및 출력시 오른쪽에 X 표시
    if predict == label:
        correct_count = correct_count + 1
    else:
        incorrect_count = incorrect_count + 1
        result = "X"
        message = "%d\t%s\t%s\t%s" %(base_index + index, predict, label, result)

        # 현재 이미지 분석 결과 출력 - 틀린것만
        print(message)

# 최종 결과 출력
print("=====================================")
print("running time:", analyzer.elapsed_time)
print("correct count:", correct_count, "\tcorrect percent:", (correct_count/analysis_count) * 100, "%")
print("incorrect count:", incorrect_count)

index	predict	label	correct
---------------------------------------
pred: ()
5
label: ()
5
pred: ()
17
label: ()
17
pred: ()
11
label: ()
11
pred: ()
5
label: ()
5
pred: ()
17
label: ()
17
pred: ()
3
label: ()
3
pred: ()
7
label: ()
7
pred: ()
6
label: ()
6
pred: ()
6
label: ()
6
pred: ()
12
label: ()
12
pred: ()
8
label: ()
8
pred: ()
10
label: ()
10
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
14
label: ()
14
pred: ()
12
label: ()
12
pred: ()
0
label: ()
0
pred: ()
5
label: ()
5
pred: ()
1
label: ()
1
pred: ()
2
label: ()
2
pred: ()
9
label: ()
9
pred: ()
17
label: ()
17
pred: ()
6
label: ()
6
pred: ()
14
label: ()
14
pred: ()
1
label: ()
1
pred: ()
13
label: ()
13
pred: ()
0
label: ()
0
pred: ()
9
label: ()
9
pred: ()
6
label: ()
6
pred: ()
12
label: ()
12
pred: ()
13
label: ()
13
pred: ()
15
label: ()
15
pred: ()
7
label: ()
7
pred: ()
3
label: ()
3
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
16
label: ()
16
pred: ()
8
label: ()
8
pred: ()
16
label: ()
16
pred:

10
label: ()
10
pred: ()
9
label: ()
9
pred: ()
3
label: ()
3
pred: ()
15
label: ()
15
pred: ()
7
label: ()
7
pred: ()
11
label: ()
11
pred: ()
5
label: ()
5
pred: ()
0
label: ()
0
pred: ()
3
label: ()
3
pred: ()
13
label: ()
13
pred: ()
8
label: ()
8
pred: ()
7
label: ()
7
pred: ()
15
label: ()
15
pred: ()
5
label: ()
5
pred: ()
15
label: ()
15
pred: ()
15
label: ()
15
pred: ()
16
label: ()
16
pred: ()
10
label: ()
10
pred: ()
14
label: ()
14
pred: ()
11
label: ()
11
pred: ()
5
label: ()
5
pred: ()
2
label: ()
2
pred: ()
12
label: ()
12
pred: ()
2
label: ()
2
pred: ()
6
label: ()
6
pred: ()
2
label: ()
2
pred: ()
2
label: ()
2
pred: ()
8
label: ()
8
pred: ()
4
label: ()
4
pred: ()
15
label: ()
15
pred: ()
6
label: ()
6
pred: ()
9
label: ()
9
pred: ()
14
label: ()
14
pred: ()
17
label: ()
17
pred: ()
2
label: ()
2
pred: ()
5
label: ()
5
pred: ()
4
label: ()
4
pred: ()
16
label: ()
16
pred: ()
8
label: ()
8
pred: ()
17
label: ()
17
pred: ()
0
label: ()
0
pred: ()
16
label: ()
16
pred: (

4
pred: ()
3
label: ()
3
pred: ()
6
label: ()
6
pred: ()
15
label: ()
15
pred: ()
17
label: ()
17
pred: ()
7
label: ()
7
pred: ()
15
label: ()
15
pred: ()
5
label: ()
5
pred: ()
2
label: ()
2
pred: ()
12
label: ()
12
pred: ()
15
label: ()
15
pred: ()
7
label: ()
7
pred: ()
8
label: ()
8
pred: ()
8
label: ()
8
pred: ()
2
label: ()
2
pred: ()
17
label: ()
17
pred: ()
5
label: ()
5
pred: ()
14
label: ()
14
pred: ()
1
label: ()
1
pred: ()
16
label: ()
16
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
9
label: ()
9
pred: ()
9
label: ()
9
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
1
label: ()
1
pred: ()
13
label: ()
13
pred: ()
15
label: ()
15
pred: ()
10
label: ()
10
pred: ()
0
label: ()
0
pred: ()
7
label: ()
7
pred: ()
6
label: ()
6
pred: ()
14
label: ()
14
pred: ()
15
label: ()
15
pred: ()
17
label: ()
17
pred: ()
14
label: ()
14
pred: ()
4
label: ()
4
pred: ()
3
label: ()
3
pred: ()
12
label: ()
12
pred: ()
5
label: ()
5
pred: ()
7
label: ()
7
pred: ()
4
label: ()
4


pred: ()
12
label: ()
12
pred: ()
3
label: ()
3
pred: ()
9
label: ()
9
pred: ()
5
label: ()
5
pred: ()
13
label: ()
13
pred: ()
14
label: ()
14
pred: ()
4
label: ()
4
pred: ()
5
label: ()
5
pred: ()
12
label: ()
12
pred: ()
3
label: ()
3
pred: ()
11
label: ()
11
pred: ()
14
label: ()
14
pred: ()
15
label: ()
15
pred: ()
12
label: ()
12
pred: ()
9
label: ()
9
pred: ()
10
label: ()
10
pred: ()
12
label: ()
12
pred: ()
16
label: ()
16
pred: ()
1
label: ()
1
pred: ()
16
label: ()
16
pred: ()
17
label: ()
17
pred: ()
4
label: ()
4
pred: ()
6
label: ()
6
pred: ()
14
label: ()
14
pred: ()
16
label: ()
16
pred: ()
11
label: ()
11
pred: ()
0
label: ()
0
pred: ()
10
label: ()
10
pred: ()
17
label: ()
17
pred: ()
12
label: ()
12
pred: ()
14
label: ()
14
pred: ()
0
label: ()
0
pred: ()
14
label: ()
14
pred: ()
13
label: ()
13
pred: ()
17
label: ()
17
pred: ()
15
label: ()
15
pred: ()
15
label: ()
15
pred: ()
10
label: ()
10
pred: ()
17
label: ()
17
pred: ()
10
label: ()
10
pred: ()
5
label: ()
5
p

7
label: ()
7
pred: ()
5
label: ()
5
pred: ()
1
label: ()
1
pred: ()
16
label: ()
16
pred: ()
1
label: ()
1
pred: ()
16
label: ()
16
pred: ()
17
label: ()
17
pred: ()
0
label: ()
0
pred: ()
5
label: ()
5
pred: ()
7
label: ()
7
pred: ()
15
label: ()
15
pred: ()
10
label: ()
10
pred: ()
17
label: ()
17
pred: ()
2
label: ()
2
pred: ()
16
label: ()
16
pred: ()
15
label: ()
15
pred: ()
2
label: ()
2
pred: ()
1
label: ()
1
pred: ()
13
label: ()
13
pred: ()
14
label: ()
14
pred: ()
7
label: ()
7
pred: ()
0
label: ()
0
pred: ()
4
label: ()
4
pred: ()
0
label: ()
0
pred: ()
11
label: ()
11
pred: ()
2
label: ()
2
pred: ()
11
label: ()
11
pred: ()
17
label: ()
17
pred: ()
9
label: ()
9
pred: ()
5
label: ()
5
pred: ()
15
label: ()
15
pred: ()
11
label: ()
11
pred: ()
16
label: ()
16
pred: ()
0
label: ()
0
pred: ()
16
label: ()
16
pred: ()
1
label: ()
1
pred: ()
13
label: ()
13
pred: ()
2
label: ()
2
pred: ()
11
label: ()
11
pred: ()
7
label: ()
7
pred: ()
6
label: ()
6
pred: ()
5
label: ()
5
pred:

...

label: ()
9
pred: ()
7
label: ()
7
pred: ()
2
label: ()
2
pred: ()
15
label: ()
15
pred: ()
16
label: ()
16
pred: ()
11
label: ()
11
pred: ()
14
label: ()
14
pred: ()
14
label: ()
14
pred: ()
16
label: ()
16
pred: ()
3
label: ()
3
pred: ()
15
label: ()
15
pred: ()
14
label: ()
14
pred: ()
16
label: ()
16
pred: ()
5
label: ()
5
pred: ()
16
label: ()
16
pred: ()
0
label: ()
0
pred: ()
17
label: ()
17
pred: ()
14
label: ()
14
pred: ()
3
label: ()
3
pred: ()
10
label: ()
10
pred: ()
2
label: ()
2
pred: ()
11
label: ()
11
pred: ()
13
label: ()
13
pred: ()
13
label: ()
13
pred: ()
13
label: ()
13
pred: ()
16
label: ()
16
pred: ()
7
label: ()
7
pred: ()
8
label: ()
8
pred: ()
6
label: ()
6
pred: ()
7
label: ()
7
pred: ()
2
label: ()
2
pred: ()
13
label: ()
13
pred: ()
0
label: ()
0
pred: ()
6
label: ()
6
pred: ()
9
label: ()
9
pred: ()
2
label: ()
2
pred: ()
5
label: ()
5
pred: ()
9
label: ()
9
pred: ()
11
label: ()
11
pred: ()
0
label: ()
0
pred: ()
8
label: ()
8
pred: ()
2
label: ()
2
pred:

pred: ()
8
label: ()
8
pred: ()
2
label: ()
2
pred: ()
0
label: ()
0
pred: ()
9
label: ()
9
pred: ()
5
label: ()
5
pred: ()
17
label: ()
17
pred: ()
8
label: ()
8
pred: ()
1
label: ()
1
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
6
label: ()
6
pred: ()
15
label: ()
15
pred: ()
12
label: ()
12
pred: ()
6
label: ()
6
pred: ()
15
label: ()
15
pred: ()
9
label: ()
9
pred: ()
14
label: ()
14
pred: ()
7
label: ()
7
pred: ()
14
label: ()
14
pred: ()
12
label: ()
12
pred: ()
6
label: ()
6
pred: ()
16
label: ()
16
pred: ()
14
label: ()
14
pred: ()
12
label: ()
12
pred: ()
6
label: ()
6
pred: ()
16
label: ()
16
pred: ()
16
label: ()
16
pred: ()
4
label: ()
4
pred: ()
8
label: ()
8
pred: ()
3
label: ()
3
pred: ()
8
label: ()
8
pred: ()
4
label: ()
4
pred: ()
0
label: ()
0
pred: ()
11
label: ()
11
pred: ()
10
label: ()
10
pred: ()
4
label: ()
4
pred: ()
4
label: ()
4
pred: ()
2
label: ()
2
pred: ()
4
label: ()
4
pred: ()
9
label: ()
9
pred: ()
15
label: ()
15
pred: ()
8
label: ()
8
pred

pred: ()
14
label: ()
14
pred: ()
12
label: ()
12
pred: ()
11
label: ()
11
pred: ()
5
label: ()
5
pred: ()
16
label: ()
16
pred: ()
5
label: ()
5
pred: ()
1
label: ()
1
pred: ()
15
label: ()
15
pred: ()
6
label: ()
6
pred: ()
16
label: ()
16
pred: ()
1
label: ()
1
pred: ()
2
label: ()
2
pred: ()
8
label: ()
8
pred: ()
11
label: ()
11
pred: ()
6
label: ()
6
pred: ()
8
label: ()
8
pred: ()
2
label: ()
2
pred: ()
7
label: ()
7
pred: ()
9
label: ()
9
pred: ()
5
label: ()
5
pred: ()
15
label: ()
15
pred: ()
2
label: ()
2
pred: ()
16
label: ()
16
pred: ()
7
label: ()
7
pred: ()
8
label: ()
8
pred: ()
16
label: ()
16
pred: ()
3
label: ()
3
pred: ()
11
label: ()
11
pred: ()
13
label: ()
13
pred: ()
11
label: ()
11
pred: ()
15
label: ()
15
pred: ()
7
label: ()
7
pred: ()
1
label: ()
1
pred: ()
5
label: ()
5
pred: ()
16
label: ()
16
pred: ()
9
label: ()
9
pred: ()
12
label: ()
12
pred: ()
16
label: ()
16
pred: ()
3
label: ()
3
pred: ()
6
label: ()
6
pred: ()
3
label: ()
3
pred: ()
6
label: ()
6


pred: ()
5
label: ()
5
pred: ()
6
label: ()
6
pred: ()
2
label: ()
2
pred: ()
11
label: ()
11
pred: ()
6
label: ()
6
pred: ()
12
label: ()
12
pred: ()
6
label: ()
6
pred: ()
14
label: ()
14
pred: ()
5
label: ()
5
pred: ()
11
label: ()
11
pred: ()
5
label: ()
5
pred: ()
16
label: ()
16
pred: ()
1
label: ()
1
pred: ()
9
label: ()
9
pred: ()
2
label: ()
2
pred: ()
5
label: ()
5
pred: ()
7
label: ()
7
pred: ()
4
label: ()
4
pred: ()
0
label: ()
0
pred: ()
12
label: ()
12
pred: ()
5
label: ()
5
pred: ()
4
label: ()
4
pred: ()
8
label: ()
8
pred: ()
15
label: ()
15
pred: ()
13
label: ()
13
pred: ()
3
label: ()
3
pred: ()
8
label: ()
8
pred: ()
17
label: ()
17
pred: ()
13
label: ()
13
pred: ()
10
label: ()
10
pred: ()
8
label: ()
8
pred: ()
3
label: ()
3
pred: ()
0
label: ()
0
pred: ()
14
label: ()
14
pred: ()
0
label: ()
0
pred: ()
4
label: ()
4
pred: ()
9
label: ()
9
pred: ()
12
label: ()
12
pred: ()
12
label: ()
12
pred: ()
16
label: ()
16
pred: ()
6
label: ()
6
pred: ()
4
label: ()
4
pred

label: ()
4
pred: ()
11
label: ()
11
pred: ()
13
label: ()
13
pred: ()
15
label: ()
15
pred: ()
8
label: ()
8
pred: ()
9
label: ()
9
pred: ()
4
label: ()
4
pred: ()
6
label: ()
6
pred: ()
17
label: ()
17
pred: ()
8
label: ()
8
pred: ()
1
label: ()
1
pred: ()
17
label: ()
17
pred: ()
5
label: ()
5
pred: ()
0
label: ()
0
pred: ()
7
label: ()
7
pred: ()
10
label: ()
10
pred: ()
1
label: ()
1
pred: ()
3
label: ()
3
pred: ()
12
label: ()
12
pred: ()
13
label: ()
13
pred: ()
11
label: ()
11
pred: ()
2
label: ()
2
pred: ()
7
label: ()
7
pred: ()
8
label: ()
8
pred: ()
0
label: ()
0
pred: ()
1
label: ()
1
pred: ()
9
label: ()
9
pred: ()
1
label: ()
1
pred: ()
17
label: ()
17
pred: ()
13
label: ()
13
pred: ()
13
label: ()
13
pred: ()
13
label: ()
13
pred: ()
2
label: ()
2
pred: ()
5
label: ()
5
pred: ()
12
label: ()
12
pred: ()
17
label: ()
17
pred: ()
7
label: ()
7
pred: ()
16
label: ()
16
pred: ()
11
label: ()
11
pred: ()
1
label: ()
1
pred: ()
7
label: ()
7
pred: ()
11
label: ()
11
pred: ()
